In [27]:
# Importing image processing libraries
import cv2  # OpenCV for image processing
import numpy as np  # NumPy for numerical operations

# Importing deep learning frameworks
import tensorflow as tf  # TensorFlow for building and training models

In [28]:
import os
from PIL import Image

def load_image_data(data_dir):
    image_label_pairs = []

    # Loop through each subdirectory (label folder)
    for label_dir in os.listdir(data_dir):
        if os.path.isdir(os.path.join(data_dir, label_dir)):
            # Get the label name
            label_name = label_dir

            # Get all image files in the label directory
            image_files = [f for f in os.listdir(os.path.join(data_dir, label_dir)) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

            # Process all images
            for img_file in image_files:
                # Build the image path
                img_path = os.path.join(data_dir, label_dir, img_file)

                # Append (image_path, label_name) tuple to the list
                image_label_pairs.append((img_path, label_name))

    return image_label_pairs

# Example usage
data_dir = r"C:\Users\chaitanya\Downloads\Data\dataset\Metals"
image_label_pairs = load_image_data(data_dir)

print(f"Loaded {len(image_label_pairs)} images with corresponding labels.")

# Show the actual labels
output_labels = set([pair[1] for pair in image_label_pairs])

print("\nLabels:")
for label, label_name in enumerate(output_labels):
    print(f"{label}: {label_name}")





Loaded 323 images with corresponding labels.

Labels:
0: plastic & glass fibre
1: copper
2: Brass Metal
3: steel
4: wood


In [29]:
import cv2
import numpy as np

def preprocess_images(image_label_pairs, target_size=(128, 128)):
    """
    Preprocesses a list of images by loading, resizing, and normalizing.
    
    Args:
    - image_label_pairs (list): List of tuples, where each tuple contains (image_path, label).
    - target_size (tuple): Target size to resize images (default: (128, 128)).
    
    Returns:
    - preprocessed_images (list): List of preprocessed images (numpy arrays).
    - labels (list): List of corresponding labels.
    """
    preprocessed_images = []
    labels = []

    for img_path, label in image_label_pairs:
        # Load the image in color
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # Check if the image is loaded successfully
        if img is None:
            print(f"Error loading image: {img_path}")
            continue

        # Resize the image to the target size
        img = cv2.resize(img, target_size)

        # Normalize the pixel values to the range [0, 1]
        img = img / 255.0

        # Append preprocessed image and label to the lists
        preprocessed_images.append(img)
        labels.append(label)

    return preprocessed_images, labels

# Example usage
preprocessed_images, labels = preprocess_images(image_label_pairs)

print(f"Preprocessed {len(preprocessed_images)} images with corresponding labels.")


Preprocessed 323 images with corresponding labels.


In [30]:
from sklearn.model_selection import train_test_split

# Split data into training and temp sets (70% training, 30% temp) with stratify
train_images, temp_images, train_labels, temp_labels = train_test_split(preprocessed_images, labels, test_size=0.3, stratify=labels, random_state=42)

# Split temp set into validation and test sets (50% validation, 50% test) with stratify
val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)

# Print sizes of each set
print(f"Training set: {len(train_images)} images")
print(f"Validation set: {len(val_images)} images")
print(f"Test set: {len(test_images)} images")



Training set: 226 images
Validation set: 48 images
Test set: 49 images


In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    """
    Creates a convolutional neural network (CNN) model with four convolutional layers.
    
    Args:
    - input_shape (tuple): Shape of input images (height, width, channels).
    - num_classes (int): Number of classes for classification.
    
    Returns:
    - TensorFlow Keras Sequential model: CNN model.
    """
    model = models.Sequential()

    # Convolutional layer 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 4
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten layer
    model.add(layers.Flatten())

    # Fully connected layer
    model.add(layers.Dense(512, activation='relu'))

    # Dropout regularization
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Example usage:
# Define input shape and number of classes
input_shape = (128, 128, 3)  # Example input shape for 128x128 RGB images
num_classes = len(set(labels))  # Number of unique classes based on your data

# Create CNN model
cnn_model = create_cnn_model(input_shape, num_classes)

# Compile the model
cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Print model summary
cnn_model.summary()


c:\Users\chaitanya\anaconda34\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,110,085 (19.49 MB)

 Trainable params: 5,110,085 (19.49 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels for training data
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform labels for validation and test data
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [33]:
import numpy as np

# Convert images to NumPy arrays
train_images_np = np.array(train_images)
val_images_np = np.array(val_images)
test_images_np = np.array(test_images)

# Convert labels to NumPy arrays
train_labels_np = np.array(train_labels_encoded)
val_labels_np = np.array(val_labels_encoded)
test_labels_np = np.array(test_labels_encoded)

In [36]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard



# Prepare callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)  # Stop training if val_loss doesn't improve for 5 epochs



# Train the model with callbacks
history = cnn_model.fit(train_images_np, train_labels_np, epochs=30,
                        validation_data=(val_images_np, val_labels_np),
                        callbacks=[early_stopping])  # Pass callbacks as a list

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_model.evaluate(test_images_np, test_labels_np)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 296ms/step - accuracy: 0.8128 - loss: 0.5302 - val_accuracy: 0.7292 - val_loss: 0.7292
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 282ms/step - accuracy: 0.8106 - loss: 0.5417 - val_accuracy: 0.6875 - val_loss: 1.0417
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 281ms/step - accuracy: 0.8307 - loss: 0.5584 - val_accuracy: 0.7708 - val_loss: 0.7185
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 272ms/step - accuracy: 0.8001 - loss: 0.5478 - val_accuracy: 0.7083 - val_loss: 0.7207
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 252ms/step - accuracy: 0.8726 - loss: 0.3812 - val_accuracy: 0.7500 - val_loss: 0.7645
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 272ms/step - accuracy: 0.8831 - loss: 0.3540 - val_accuracy: 0.7500 - val_loss: 0.8212
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 279ms/step - accuracy: 0.9108 - loss: 0.2556 - val_accuracy: 0.7500 - val_loss: 0.8855
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 258ms/step - accuracy: 0.9258 - loss: 0.2765 - val_accuracy: 0.7708 - val_loss:

In [37]:


# Get predictions from the model
predicted_probs = cnn_model.predict(test_images_np)  # This gives you predicted probabilities for each class
predicted_labels = np.argmax(predicted_probs, axis=1)  # Convert probabilities to class labels



# Example usage of classification_report with zero_division='warn'
from sklearn.metrics import classification_report

print(classification_report(test_labels_np, predicted_labels, zero_division='warn'))



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
              precision    recall  f1-score   support

           0       0.84      0.94      0.89        17
           1       0.50      0.50      0.50         2
           2       0.00      0.00      0.00         2
           3       0.88      0.92      0.90        24
           4       1.00      0.75      0.86         4

    accuracy                           0.86        49
   macro avg       0.64      0.62      0.63        49
weighted avg       0.83      0.86      0.84        49



c:\Users\chaitanya\anaconda34\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chaitanya\anaconda34\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chaitanya\anaconda34\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
